In [1]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        '''Adds items from an iterable'''
        
    @abc.abstractmethod
    def pick(self):
        '''Remove item at random, returning it
        This method should raise 'LookupError' when the instance
        is empty'''
        
    def loaded(self):
        '''Return True if there's at least 1 item, 'False' otherwise'''
        return bool(self.inspect())
    
    def inspect(self):
        '''Return a sorted tuple with the items
        currently inside'''
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [2]:
class Fake(Tombola):
    def pick(self):
        return 13

In [3]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

## BingoCage

In [4]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [5]:
import random

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty Lottery Blower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

In [7]:
from random import randrange

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
    # Tombola.register(TomboList) #

In [8]:
issubclass(TomboList, Tombola)

True

In [9]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [13]:
t.pick()

86